### Text classification  using RNN

In [1]:
from __future__ import division, print_function
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
import re
import nltk
vfrom nltk import word_tokenize
from gensim import models
import gensim.downloader as api
from nltk.corpus import stopwords

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']
nltk.download('stopwords')


Using TensorFlow backend.


Load the dataset

In [2]:
data = pd.read_csv ('hotel _eviews.csv',  engine='python')   
print (data.head)

<bound method NDFrame.head of                          Property Name  Review_Rating  \
0               Apex London Wall Hotel              5   
1               Corinthia Hotel London              5   
2                            The Savoy              5   
3                         Rhodes Hotel              4   
4                            The Savoy              5   
..                                 ...            ...   
967                     The Dorchester              5   
968             Apex London Wall Hotel              4   
969                      The Rembrandt              3   
970  Hotel Xenia, Autograph Collection              4   
971             Corinthia Hotel London              5   

                                     Review_Title  \
0                           Ottima qualità prezzo   
1              By far, my best hotel in the world   
2    First visit to the American Bar at the Savoy   
3                                       Nice stay   
4                   

Check if it has missing values

In [3]:
data.isna().sum()



Property Name                 0
Review_Rating                 0
Review_Title                  0
Review_Text                   0
Location Of The Reviewer    102
Date Of Review                0
dtype: int64

Drop unnecessary columns

In [4]:
data= data.drop('Location Of The Reviewer', 1)
data= data.drop('Date Of Review', 1)


In [5]:
print (data.head)

<bound method NDFrame.head of                          Property Name  Review_Rating  \
0               Apex London Wall Hotel              5   
1               Corinthia Hotel London              5   
2                            The Savoy              5   
3                         Rhodes Hotel              4   
4                            The Savoy              5   
..                                 ...            ...   
967                     The Dorchester              5   
968             Apex London Wall Hotel              4   
969                      The Rembrandt              3   
970  Hotel Xenia, Autograph Collection              4   
971             Corinthia Hotel London              5   

                                     Review_Title  \
0                           Ottima qualità prezzo   
1              By far, my best hotel in the world   
2    First visit to the American Bar at the Savoy   
3                                       Nice stay   
4                   

Here we change review rating into 0 and 1

In [5]:
data['Review_Rating'] = data['Review_Rating'].replace(1, 0)
data['Review_Rating'] = data['Review_Rating'].replace(2, 0)
data['Review_Rating'] = data['Review_Rating'].replace(3, 0)
data['Review_Rating'] = data['Review_Rating'].replace(4, 1)
data['Review_Rating'] = data['Review_Rating'].replace(5, 1)



In [7]:
print (data.head)

<bound method NDFrame.head of                          Property Name  Review_Rating  \
0               Apex London Wall Hotel              1   
1               Corinthia Hotel London              1   
2                            The Savoy              1   
3                         Rhodes Hotel              1   
4                            The Savoy              1   
..                                 ...            ...   
967                     The Dorchester              1   
968             Apex London Wall Hotel              1   
969                      The Rembrandt              0   
970  Hotel Xenia, Autograph Collection              1   
971             Corinthia Hotel London              1   

                                     Review_Title  \
0                           Ottima qualità prezzo   
1              By far, my best hotel in the world   
2    First visit to the American Bar at the Savoy   
3                                       Nice stay   
4                   

Check the polarity of the data

In [6]:
data.Review_Rating.value_counts()


1    872
0    100
Name: Review_Rating, dtype: int64

In [7]:
data.dtypes

Property Name    object
Review_Rating     int64
Review_Title     object
Review_Text      object
dtype: object

In [9]:
pos = []
neg = []
for l in data.Review_Rating:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [10]:
data['Pos']= pos
data['Neg']= neg

In [13]:
data.head()


,Property Name,Review_Rating,Review_Title,Review_Text,Pos,Neg
0,Apex London Wall Hotel,1,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,1,0
1,Corinthia Hotel London,1,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,1,0
2,The Savoy,1,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,1,0
3,Rhodes Hotel,1,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,1,0
4,The Savoy,1,Perfection,Form the moment we arrived until we left we ex...,1,0


In [11]:
data.shape

(972, 6)

In [ ]:
Remove the punctuation marks 

In [12]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Review_Text'].apply(lambda x: remove_punct(x))

In [13]:
data.Text_Clean

0      Siamo stati a Londra per un week end ed abbiam...
1      I had a pleasure of staying in this hotel for ...
2      A very lovely first visit to this iconic hotel...
3      3 of us stayed at the Rhodes Hotel for 4 night...
4      Form the moment we arrived until we left we ex...
                             ...                        
967    As always we had a great time at The Dorcheste...
968    Hotel liegt etwas versteckt und daher auch für...
969    Hôtel idéalement situé pour bien visiter pas m...
970    I requested a terrace room it wasnt really a t...
971    Great for the clever traveller We booked three...
Name: Text_Clean, Length: 972, dtype: object

In [14]:
tokens = [word_tokenize(sen) for sen in data.Text_Clean]  #tokenize the data

Then make all words in tokens in lowercase

In [18]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

Removing Stop words

In [19]:
stoplist = stopwords.words('english')
def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [removeStopWords(sen) for sen in lower_tokens]
data['Text_Final'] = [' '.join(sen) for sen in filtered_words]
data['tokens'] = filtered_words

In [20]:
data.head()

,Property Name,Review_Rating,Review_Title,Review_Text,Pos,Neg,Text_Clean,Text_Final,tokens
0,Apex London Wall Hotel,1,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,1,0,Siamo stati a Londra per un week end ed abbiam...,siamo stati londra per un week end ed abbiamo ...,"[siamo, stati, londra, per, un, week, end, ed,..."
1,Corinthia Hotel London,1,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,1,0,I had a pleasure of staying in this hotel for ...,pleasure staying hotel 7 nights recently hotel...,"[pleasure, staying, hotel, 7, nights, recently..."
2,The Savoy,1,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,1,0,A very lovely first visit to this iconic hotel...,lovely first visit iconic hotel bar wonderful ...,"[lovely, first, visit, iconic, hotel, bar, won..."
3,Rhodes Hotel,1,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,1,0,3 of us stayed at the Rhodes Hotel for 4 night...,3 us stayed rhodes hotel 4 nights great locati...,"[3, us, stayed, rhodes, hotel, 4, nights, grea..."
4,The Savoy,1,Perfection,Form the moment we arrived until we left we ex...,1,0,Form the moment we arrived until we left we ex...,form moment arrived left experienced absolute ...,"[form, moment, arrived, left, experienced, abs..."


Split data into test and train


In [22]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.10, 
                                         random_state=42)

let's see how many words we have and maximum sentence length in training dataset

In [23]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

63098 words total, with a vocabulary size of 10820
Max sentence length is 771


let's see how many words we have and maximum sentence length in test dataset

In [24]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

7045 words total, with a vocabulary size of 2481
Max sentence length is 352


Now we should convert the words into vector by embadding method such as word2vec

In [ ]:
#Define paramthers
MAX_SEQUENCE_LENGTH = 50 # the lenth of words in sentence
EMBEDDING_DIM = 300

In [32]:
word2vec = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [34]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 10820 unique tokens.


In [61]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(10821, 300)


let's see how it looks

In [64]:
train_embedding_weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02612305,  0.0144043 , -0.03442383, ...,  0.15625   ,
         0.09960938,  0.12695312],
       [ 0.02624512,  0.1328125 , -0.0234375 , ...,  0.12158203,
        -0.1328125 ,  0.10302734],
       ...,
       [-0.09472656, -0.15429688, -0.26367188, ..., -0.10644531,
         0.04345703,  0.12255859],
       [-0.14648438, -0.18457031,  0.11279297, ...,  0.29492188,
         0.12597656, -0.11083984],
       [ 0.25820179,  0.68346509,  0.52196713, ...,  0.62167443,
         0.93203331,  0.20247044]])

In [73]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_RNN_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_RNN_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)


After padding please refer to padding in other reference

In [71]:
test_RNN_data

array([[ 637,    4,    9, ...,  494,  624,    5],
       [   0,    0,    0, ...,   48,  192,    4],
       [ 379,   58,  380, ..., 2738, 2794,   23],
       ...,
       [   0,    0,    0, ...,  900,   21,    1],
       [ 395,    4,   34, ...,    8,  848,    5],
       [1088,  692,  137, ...,  118,   54, 1088]])

In [74]:
Rating_names = ['Pos', 'Neg']
input_train = train_RNN_data  # input of the neural nework
output_train = data_train[Rating_names].values  # output of neural network



We will build model based on RNN and LSTM

In [45]:
def rnn(embeddings, 
        max_sequence_length, 
        num_words, 
        embedding_dim, 
        labels_index):
    
    embedding_layer = Embedding(num_words,
                                embedding_dim,
                                weights=[embeddings],
                                input_length=max_sequence_length,
                                trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,),
                                  dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [53]:
model = rnn(train_embedding_weights, 
            MAX_SEQUENCE_LENGTH, 
            len(train_word_index)+1, 
            EMBEDDING_DIM, 
            len(list(Rating_names)))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 300)           3246300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 3,849,822
Trainable params: 603,522
Non-trainable params: 3,246,300
___________________________________________

In [76]:
#Train the model by our dataset
num_iteration = 10
batch_size = 34
hist = model.fit(input_train, output_train, epochs=num_iteration, validation_split=0.1, shuffle=True, batch_size=batch_size)


Train on 786 samples, validate on 88 samples
Epoch 1/10
786/786 [==============================] - 57s 73ms/step - loss: 0.2073 - acc: 0.9256 - val_loss: 0.2432 - val_acc: 0.8977
Epoch 2/10
786/786 [==============================] - 56s 71ms/step - loss: 0.1987 - acc: 0.9249 - val_loss: 0.2921 - val_acc: 0.9034
Epoch 3/10
786/786 [==============================] - 55s 70ms/step - loss: 0.2440 - acc: 0.9065 - val_loss: 0.2192 - val_acc: 0.8977
Epoch 4/10
786/786 [==============================] - 56s 71ms/step - loss: 0.1452 - acc: 0.9440 - val_loss: 0.2499 - val_acc: 0.9091
Epoch 5/10
786/786 [==============================] - 59s 75ms/step - loss: 0.1332 - acc: 0.9542 - val_loss: 0.2232 - val_acc: 0.9148
Epoch 6/10
786/786 [==============================] - 62s 79ms/step - loss: 0.1547 - acc: 0.9478 - val_loss: 0.2900 - val_acc: 0.8864
Epoch 7/10
786/786 [==============================] - 65s 83ms/step - loss: 0.1003 - acc: 0.9663 - val_loss: 0.2519 - val_acc: 0.9091
Epoch 8/10
786/78

In [77]:
predictions = model.predict(test_RNN_data, batch_size=1024, verbose=1)


98/98 [==============================] - 2s 24ms/step


In [78]:
labels = [1, 0]


In [79]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [80]:
sum(data_test.Review_Rating==prediction_labels)/len(prediction_labels)


0.8673469387755102

So our prediction is about 87% which consider good for a small dataset that we used